In [1]:
import pandas as pd
import category_encoders as ce

divar_1 = pd.read_csv('Divar.csv')
divar_2 = divar_1[sorted(divar_1.columns)]
pd.set_option('display.max_rows', None)  # نمایش تمامی سطرها
pd.set_option('display.max_columns', None)  # نمایش تمامی ستون‌ها
pd.set_option('display.expand_frame_repr', False)  # جلوگیری از خلاصه شدن
pd.set_option('max_colwidth', None)

D:\AppData\ipykernel_40808\822924179.py:4: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  divar_1 = pd.read_csv('Divar.csv')


In [2]:
divar_2.shape

(1000000, 61)

In [3]:
desired_columns = [
    'building_direction',
    'building_size',
    'cat2_slug',
    'cat3_slug',
    'city_slug',
    'construction_year',
    'created_at_month',
    'deed_type',
    'has_balcony',
    'has_elevator',
    'has_barbecue',
    'has_jacuzzi',
    'has_parking',
    'has_pool',
    'has_sauna',
    'has_security_guard',
    'has_warehouse',
    'is_rebuilt',
    'land_size',
    'neighborhood_slug',
    'price_value',
    'rooms_count'
]

In [4]:
divar_3 = divar_2[desired_columns].copy()

In [5]:
divar_3.shape

(1000000, 22)

آماده سازی ستون building_direction

In [6]:
divar_3['building_direction'].value_counts()

building_direction
south       159453
north       142009
east         12111
west          7143
unselect      3207
Name: count, dtype: int64

In [7]:
divar_3["building_direction_missing"] = divar_3["building_direction"].isnull().astype(int)

In [8]:
divar_3['building_direction'] = divar_3['building_direction'].fillna('unknown')

In [9]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_building_direction = encoder.fit_transform(divar_3[["building_direction"]])
encoded_buiding_direction_df = pd.DataFrame(encoded_building_direction, columns=encoder.get_feature_names_out(["building_direction"]))
divar_4 = pd.concat([divar_3, encoded_buiding_direction_df], axis=1).drop(columns=["building_direction"])



آماده سازی ستون building_size

In [10]:
median_building_size = divar_4['building_size'].median()
divar_4['building_size'].fillna(median_building_size, inplace=True)

D:\AppData\ipykernel_40808\3468348801.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_4['building_size'].fillna(median_building_size, inplace=True)


آماده سازی ستون construction_year

In [11]:
divar_4.loc[divar_4["construction_year"] == "قبل از ۱۳۷۰", "construction_year"] = "۱۳۷۰"


In [12]:
divar_4['construction_year'] = divar_4['construction_year'].astype(str).str.translate(str.maketrans('۰۱۲۳۴۵۶۷۸۹', '0123456789'))


In [13]:
divar_4['construction_year'] = pd.to_numeric(divar_4['construction_year'], errors='coerce')

In [14]:
divar_4['construction_year_is_missing'] = divar_4['construction_year'].isnull().astype(int)

In [15]:
# divar_4.drop(columns=['construction_year_is_missng'], axis=1, inplace=True)

In [16]:
median_year = divar_4['construction_year'].median()
divar_4['construction_year'].fillna(median_year, inplace=True)

D:\AppData\ipykernel_40808\879027852.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_4['construction_year'].fillna(median_year, inplace=True)


In [17]:
divar_5 = divar_4.copy()

<p dir=rtl>
<font size=5 color=yellow>
آماده سازی ستون created_at_month

In [18]:
divar_5['created_at_month'] = divar_5['created_at_month'].str.replace(r'00:00:00', '', regex=True)

In [19]:
divar_5.rename(columns={'created_at_month' : 'creation_date'}, inplace=True)

In [20]:
# divar_5 = pd.get_dummies(divar_5, columns=['city'])

In [21]:
encoder_binary_month = ce.BinaryEncoder(cols=['creation_date'])
divar_5 = encoder_binary_month.fit_transform(divar_5)
# 

آماده سازی ستون deed_type

In [22]:
divar_6 = divar_5.copy()

In [23]:
divar_6['deed_type_is_missing'] = divar_6['deed_type'].isnull().astype(int)

In [24]:
divar_6['deed_type'] = divar_6['deed_type'].fillna('unknown')

In [25]:
encoder_6 = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_deed_type = encoder_6.fit_transform(divar_6[["deed_type"]])
encoded_deed_type_df = pd.DataFrame(encoded_deed_type, columns=encoder_6.get_feature_names_out(["deed_type"]))
divar_7 = pd.concat([divar_6, encoded_deed_type_df], axis=1).drop(columns=["deed_type"])

In [26]:
divar_7['has_balcony'].value_counts()

has_balcony
true        392096
false        88855
True         20692
False         4545
unselect       223
Name: count, dtype: int64

In [27]:
divar_7_5 = divar_7.copy()
divar_7_5['has_balcony'].replace('unselect', pd.NA , inplace=True)
divar_7_5['has_balcony'].replace(False, 'false' , inplace=True)
divar_7_5['has_balcony'].replace(True , 'true' , inplace=True)

D:\AppData\ipykernel_40808\951201462.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_7_5['has_balcony'].replace('unselect', pd.NA , inplace=True)
D:\AppData\ipykernel_40808\951201462.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].meth

In [28]:
divar_7_5['has_balcony'].value_counts()

has_balcony
true     412788
false     93400
Name: count, dtype: int64

In [29]:
divar_7_5['has_balcony_is_missing'] = divar_7_5['has_balcony'].isnull().astype(int)

In [30]:
divar_7_5['has_balcony'] = divar_7_5['has_balcony'].fillna('unselect')

In [31]:
encoder_7 = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_has_balcony = encoder_7.fit_transform(divar_7_5[["has_balcony"]])
encoded_has_balcony_df = pd.DataFrame(encoded_has_balcony, columns=encoder_7.get_feature_names_out(['has_balcony']))
divar_7_5 = pd.concat([divar_7_5, encoded_has_balcony_df], axis=1).drop(columns=['has_balcony'])



<p dir=rtl>
<font size=5 color=yellow>
آماده سازی ستون luxury_features که از ادغام ستون های has_security_guard و has_pool و has_sauna و has_barbecue و has_jacuzzi ایجاد شده است.

In [32]:
divar_7_5['has_pool'].value_counts(), divar_7_5['has_security_guard'].value_counts(), divar_7_5['has_sauna'].value_counts(), divar_7_5['has_jacuzzi'].value_counts(), divar_7_5['has_barbecue'].value_counts()

(has_pool
 False    25669
 True      3721
 Name: count, dtype: int64,
 has_security_guard
 False    24631
 True      6681
 Name: count, dtype: int64,
 has_sauna
 False    27084
 True      1395
 Name: count, dtype: int64,
 has_jacuzzi
 False    26455
 True      2273
 Name: count, dtype: int64,
 has_barbecue
 False    23377
 True      7821
 Name: count, dtype: int64)

In [33]:
divar_8 = divar_7_5.copy()

In [34]:
luxury_cols = ["has_barbecue", "has_jacuzzi", "has_pool", "has_sauna", "has_security_guard"]
divar_8[luxury_cols] = divar_8[luxury_cols].replace({True: "1", False: "0"})

In [35]:
divar_8[luxury_cols] = divar_8[luxury_cols].fillna('0')
divar_8[luxury_cols] = divar_8[luxury_cols].astype(int)
## luxury_features : luxury_features_counts
divar_8['luxury_features'] = divar_8[luxury_cols].sum(axis=1)


In [36]:
divar_8_1 = divar_8.drop(columns=['has_pool','has_jacuzzi','has_sauna','has_barbecue','has_security_guard'], axis=1)

<p dir=rtl>
<font size=5 color = yellow>
آماده سازی ستون has_elevator

In [37]:
divar_9 = divar_8_1.copy()

In [38]:
divar_9['has_elevator'].replace(False, 'false' , inplace=True)
divar_9['has_elevator'].replace(True , 'true' , inplace=True)

D:\AppData\ipykernel_40808\3625838816.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_9['has_elevator'].replace(False, 'false' , inplace=True)
D:\AppData\ipykernel_40808\3625838816.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [39]:
divar_9['has_elevator'] = divar_9['has_elevator'].fillna('unselect')

In [40]:
encoder_9 = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_has_elevator = encoder_9.fit_transform(divar_9[["has_elevator"]])
encoded_has_elevator_df = pd.DataFrame(encoded_has_elevator, columns=encoder_9.get_feature_names_out(['has_elevator']))
divar_9 = pd.concat([divar_9, encoded_has_elevator_df], axis=1).drop(columns=['has_elevator'])



<p dir=rtl>
<font size=5 color = yellow>
آماده سازی ستون has_parking

In [41]:
divar_10 = divar_9.copy()

In [42]:
divar_10['has_parking'].replace(False, 'false' , inplace=True)
divar_10['has_parking'].replace(True , 'true' , inplace=True)

D:\AppData\ipykernel_40808\4274835166.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_10['has_parking'].replace(False, 'false' , inplace=True)
D:\AppData\ipykernel_40808\4274835166.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [43]:
divar_10['has_parking'] = divar_10['has_parking'].fillna('unselect')

In [44]:
encoder_10 = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_has_parking = encoder_10.fit_transform(divar_10[["has_parking"]])
encoded_has_parking_df = pd.DataFrame(encoded_has_parking, columns=encoder_10.get_feature_names_out(['has_parking']))
divar_10 = pd.concat([divar_10, encoded_has_parking_df], axis=1).drop(columns=['has_parking'])



<p dir=rtl>
<font size=5 color = yellow>
آماده سازی ستون has_warehouse

In [45]:
divar_11 = divar_10.copy()

In [46]:
divar_11['has_warehouse'].replace(False, 'false' , inplace=True)
divar_11['has_warehouse'].replace(True , 'true' , inplace=True)

D:\AppData\ipykernel_40808\257579060.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_11['has_warehouse'].replace(False, 'false' , inplace=True)
D:\AppData\ipykernel_40808\257579060.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [47]:
divar_11['has_warehouse'] = divar_11['has_warehouse'].fillna('unselect')

In [48]:
encoder_11 = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_has_warehouse = encoder_11.fit_transform(divar_11[["has_warehouse"]])
encoded_has_warehouse_df = pd.DataFrame(encoded_has_warehouse, columns=encoder_11.get_feature_names_out(['has_warehouse']))
divar_11 = pd.concat([divar_11, encoded_has_warehouse_df], axis=1).drop(columns=['has_warehouse'])


<p dir=rtl>
<font size=5 color=yellow>
آماده سازی ستون is_rebuilt

In [49]:
divar_12 = divar_11.copy()

In [50]:
divar_12['is_rebuilt'].replace(False, 'false' , inplace=True)
divar_12['is_rebuilt'].replace(True , 'true' , inplace=True)

D:\AppData\ipykernel_40808\2060642059.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_12['is_rebuilt'].replace(False, 'false' , inplace=True)
D:\AppData\ipykernel_40808\2060642059.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(

In [51]:
divar_12['is_rebuilt'] = divar_12['is_rebuilt'].fillna('unselect')

In [52]:
encoder_12 = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_is_rebuilt = encoder_12.fit_transform(divar_12[['is_rebuilt']])
encoded_is_rebuilt_df = pd.DataFrame(encoded_is_rebuilt, columns=encoder_12.get_feature_names_out(['is_rebuilt']))
divar_12 = pd.concat([divar_12, encoded_is_rebuilt_df], axis=1).drop(columns=['is_rebuilt'])


<p dir=rtl>
<font size=5 color = yellow>
آماده سازی ستون rooms_count

In [53]:
divar_13 = divar_12.copy()

In [54]:
divar_13['rooms_count'].value_counts()

rooms_count
دو              404050
یک              192083
سه              138633
بدون اتاق        75898
چهار             21371
پنج یا بیشتر     13864
Name: count, dtype: int64

In [55]:
divar_13['rooms_count'].replace('بدون اتاق', '0',  inplace=True)
divar_13['rooms_count'].replace('یک','1',  inplace=True)
divar_13['rooms_count'].replace('دو','2',  inplace=True)
divar_13['rooms_count'].replace('سه','3',  inplace=True)
divar_13['rooms_count'].replace('چهار', '4',  inplace=True)
divar_13['rooms_count'].replace('پنج یا بیشتر', '5 or more',  inplace=True)

D:\AppData\ipykernel_40808\1911240802.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_13['rooms_count'].replace('بدون اتاق', '0',  inplace=True)
D:\AppData\ipykernel_40808\1911240802.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].meth

In [56]:
divar_13['rooms_count'] = divar_13['rooms_count'].fillna('unselect')

In [57]:
encoder_13 = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_rooms_count = encoder_13.fit_transform(divar_13[['rooms_count']])
encoded_rooms_count_df = pd.DataFrame(encoded_rooms_count, columns=encoder_13.get_feature_names_out(['rooms_count']))
divar_13 = pd.concat([divar_13, encoded_rooms_count_df], axis=1).drop(columns=['rooms_count'])


<p dir=rtl>
<font size=5 color=yellow>
آماده سازی ستون neighborhood_slug 
<br>
تغییر اسم ستون به neighborhood

In [58]:
divar_14 = divar_13.copy()

In [59]:
divar_14.loc[divar_14['neighborhood_slug'].isna(), 'neighborhood_slug'] = divar_14['city_slug']

In [60]:
divar_14.rename(columns={'neighborhood_slug' : 'neighborhood'}, inplace=True)

<p dir=rtl>
<font size=5 color=yellow>
تغییر اسم ستون city_slug به city

In [61]:
divar_14.rename(columns={'city_slug' : 'city'}, inplace=True)

<p dir=rtl>
<font size=5 color=yellow>
آماده سازی ستون land_size

In [62]:
divar_14['land_size'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1000000 entries, 0 to 999999
Series name: land_size
Non-Null Count   Dtype  
--------------   -----  
186396 non-null  float64
dtypes: float64(1)
memory usage: 7.6 MB


In [63]:
divar_14['land_size_is_missing'] = divar_14['land_size'].isnull().astype(int)

In [64]:
divar_15 = divar_14.copy()

In [65]:
median_land_size = divar_15['land_size'].median()

In [66]:
divar_15['land_size'] = divar_15['land_size'].fillna(median_land_size)

<p dir=rtl>
<font size=5 color=red>
دیتا فریم divar_15 بر اساس price_value ساخته شده اما سطرهای آن تغییر نکرده 

<p dir=rtl>
<font size = 5 color = yellow>
افزودین ستون های مربوط به اجاره به دیتافریم ساخته شده جدید

In [67]:
divar_final = divar_15.copy()

In [68]:
divar_final[['credit_value','rent_value','rent_credit_transform','transformable_credit','transformable_rent','transformable_price']] = divar_2[['credit_value','rent_value','rent_credit_transform','transformable_credit','transformable_rent','transformable_price']]

In [69]:
# divar_final.to_csv('divar_final.csv', index=False)

In [70]:
divar_15.shape

(1000000, 54)

<p dir=rtl>
<font size=5 color=yellow>
مرتب سازی بر اساس price 

In [71]:
divar_price = divar_15.copy()

In [72]:
divar_price = divar_price.dropna(subset=['price_value'])

In [73]:
divar_price = divar_price.dropna(subset=['city'])

In [74]:
divar_price['cat2_slug'].value_counts()

cat2_slug
residential-sell        534963
commercial-sell          32127
real-estate-services      1210
commercial-rent             34
residential-rent             6
temporary-rent               5
Name: count, dtype: int64

<p dir=rtl>
<font size=5 color=yellow>
نگه داشتن سطرهایی از cat2_slug که مربوط به فروش هستند.

In [75]:
divar_price_2 = divar_price[divar_price['cat2_slug'].str.contains('sell', na=False)]

In [76]:
divar_price_2.rename(columns={'cat2_slug' : 'dealing_type'}, inplace=True)

D:\AppData\ipykernel_40808\2298980766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  divar_price_2.rename(columns={'cat2_slug' : 'dealing_type'}, inplace=True)


In [77]:
divar_price_2.rename(columns={'cat3_slug' : 'building_type'}, inplace=True)

D:\AppData\ipykernel_40808\2715472140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  divar_price_2.rename(columns={'cat3_slug' : 'building_type'}, inplace=True)


In [78]:
divar_price_2.rename(columns={'dealing_type' : 'property_type'}, inplace=True)

D:\AppData\ipykernel_40808\2750678498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  divar_price_2.rename(columns={'dealing_type' : 'property_type'}, inplace=True)


<p dir=rtl>
<font size=5 color=yellow>
تبدیل تایپ ستون های object به عدد با استفاده از binary_encoder

In [79]:
encoder_binary = ce.BinaryEncoder(cols=['city','neighborhood','building_type','property_type'])
divar_price_3 = encoder_binary.fit_transform(divar_price_2)


In [80]:
divar_price_3.reset_index(drop=True, inplace=True)


In [81]:
divar_price_final = divar_price_3.copy()

In [ ]:
# divar_price_final.to_csv('divar_price_final.csv', index=False)

In [83]:
divar_price_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567090 entries, 0 to 567089
Data columns (total 75 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   building_size                 567090 non-null  float64
 1   property_type_0               567090 non-null  int64  
 2   property_type_1               567090 non-null  int64  
 3   building_type_0               567090 non-null  int64  
 4   building_type_1               567090 non-null  int64  
 5   building_type_2               567090 non-null  int64  
 6   city_0                        567090 non-null  int64  
 7   city_1                        567090 non-null  int64  
 8   city_2                        567090 non-null  int64  
 9   city_3                        567090 non-null  int64  
 10  city_4                        567090 non-null  int64  
 11  city_5                        567090 non-null  int64  
 12  city_6                        567090 non-nul